In [ ]:
!pip install filterpy

In [1]:
import cusignal
import cupy as cp # CuPy version 8.0.0b4+ required
import numpy as np
import filterpy.kalman

### Filter Size and Parameters

In [2]:
dim_x = 4
dim_z = 2
iterations = 10
tracks = 32
dt = np.float64

### State Transition Matrix

In [3]:
F = np.array(
        [
            [1.0, 0.0, 1.0, 0.0],  # x = x0 + v_x*dt
            [0.0, 1.0, 0.0, 1.0],  # y = y0 + v_y*dt
            [0.0, 0.0, 1.0, 0.0],  # dx = v_x
            [1.0, 0.0, 0.0, 1.0],
        ],  # dy = v_y
        dtype=dt,
    )

### Measurement Function

In [4]:
H = np.array(
        [
            [1.0, 0.0, 1.0, 0.0], 
            [0.0, 1.0, 0.0, 1.0]
        ],
        dtype=dt,  # x_0  # y_0
    )

### Initial Location

In [5]:
x = np.array([[10.0, 10.0, 0.0, 0.0]], dtype=dt).T  # x, y, v_x, v_y

### Initial Estimate Error

In [6]:
P = np.eye(dim_x, dtype=dt) * np.array([1.0, 1.0, 2.0, 2.0], dtype=dt)

### Measurement Noise

In [7]:
R = np.eye(dim_z, dtype=dt) * 0.01

### Motion Noise

In [8]:
Q = np.eye(dim_x, dtype=dt) * np.array([10.0, 10.0, 10.0, 10.0], dtype=dt)

## Process CPU Kalman Filter
For CPU example, each track is ran over the course of all iterations

In [9]:
f_fpy = filterpy.kalman.KalmanFilter(dim_x=dim_x, dim_z=dim_z)

for _ in range(tracks):
    np.random.seed(1234)
    f_fpy.x = x
    f_fpy.F = F
    f_fpy.H = H
    f_fpy.P = P
    f_fpy.R = R
    f_fpy.Q = Q
    
    for _ in range(iterations):
        f_fpy.predict()
        z = np.random.random_sample(dim_z).astype(dt).T
        f_fpy.update(z)

## Process GPU Kalman Filter
Initial data for all tracks

In [10]:
cuS = cusignal.KalmanFilter(dim_x, dim_z, points=tracks, dtype=dt)

cuS.x = cp.repeat(cp.asarray(x[cp.newaxis, :, :]), tracks, axis=0)
cuS.F = cp.repeat(cp.asarray(F[cp.newaxis, :, :]), tracks, axis=0)
cuS.H = cp.repeat(cp.asarray(H[cp.newaxis, :, :]), tracks, axis=0)
cuS.P = cp.repeat(cp.asarray(P[cp.newaxis, :, :]), tracks, axis=0)
cuS.R = cp.repeat(cp.asarray(R[cp.newaxis, :, :]), tracks, axis=0)
cuS.Q = cp.repeat(cp.asarray(Q[cp.newaxis, :, :]), tracks, axis=0)

np.random.seed(1234)

In [11]:
for _ in range(iterations):
    cuS.predict()
    
    z = np.atleast_2d(np.random.random_sample(dim_z).astype(dt)).T
    z = np.repeat(z[np.newaxis, :, :], tracks, axis=0)
    
    cuS.update(z)
    
cp.cuda.runtime.deviceSynchronize()

### Test Results
We compare the first and last cuSignal estimate with the CPU version

In [12]:
np.testing.assert_allclose(f_fpy.x, cuS.x[0, :, :].get())
np.testing.assert_allclose(f_fpy.x, cuS.x[-1, :, :].get())

In [13]:
np.testing.assert_allclose(f_fpy.P, cuS.P[0, :, :].get())
np.testing.assert_allclose(f_fpy.P, cuS.P[-1, :, :].get())

## Benchmarks

In [14]:
iterations = 50
tracks = 2**12

In [15]:
f_fpy = filterpy.kalman.KalmanFilter(dim_x=dim_x, dim_z=dim_z)

CPU version may take a few minutes to run

In [16]:
%%timeit
for _ in range(tracks):
    np.random.seed(1234)
    f_fpy.x = x
    f_fpy.F = F
    f_fpy.H = H
    f_fpy.P = P
    f_fpy.R = R
    f_fpy.Q = Q
    
    for i in range(iterations):
        f_fpy.predict()
        z = np.ones(dim_z).astype(dt).T * i
        f_fpy.update(z)

10.4 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
cuS = cusignal.KalmanFilter(dim_x, dim_z, points=tracks, dtype=dt)

cuS.x = cp.repeat(cp.asarray(x[cp.newaxis, :, :]), tracks, axis=0)
cuS.F = cp.repeat(cp.asarray(F[cp.newaxis, :, :]), tracks, axis=0)
cuS.H = cp.repeat(cp.asarray(H[cp.newaxis, :, :]), tracks, axis=0)
cuS.P = cp.repeat(cp.asarray(P[cp.newaxis, :, :]), tracks, axis=0)
cuS.R = cp.repeat(cp.asarray(R[cp.newaxis, :, :]), tracks, axis=0)
cuS.Q = cp.repeat(cp.asarray(Q[cp.newaxis, :, :]), tracks, axis=0)

np.random.seed(1234)

In [18]:
%%timeit
for i in range(iterations):
    cuS.predict()
    
    z = np.atleast_2d(np.ones(dim_z).astype(dt)).T * i
    z = np.repeat(z[np.newaxis, :, :], tracks, axis=0)
    
    cuS.update(z)
    
cp.cuda.runtime.deviceSynchronize()

4.99 ms ± 36.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
np.testing.assert_allclose(f_fpy.x, cuS.x[0, :, :].get())
np.testing.assert_allclose(f_fpy.x, cuS.x[-1, :, :].get())

In [20]:
np.testing.assert_allclose(f_fpy.P, cuS.P[0, :, :].get())
np.testing.assert_allclose(f_fpy.P, cuS.P[-1, :, :].get())